In [1]:
import numpy as np
import pandas as pd
import sys
import getpass
import re
from functions import *
from collections import Counter
import copy
np.set_printoptions(precision=2)
pd.set_option('precision', 2)
%matplotlib inline
matplotlib.style.use('ggplot')

In [2]:
PATH = '/Google Drive/Jonathan Sarah Ido folder/data/CCK/'
def get_path(path = PATH):
    if os.name == 'posix':
        return os.environ['HOME']+path #'/Google Drive/Jonathan Sarah Ido folder/data/CCK/'
    elif os.name == 'nt':
        return os.environ['USERPROFILE']+ path.replace('/','\\') #'\\Google Drive\Jonathan Sarah Ido folder\data\CCK\\'
    else:
        raise Exception('OS not recongnized. I\'m confused.')
gitpath = '/Documents/git/Phet-log-analyzer/cck/raw_data_parsing_check/'
df = pd.read_csv(get_path(path = gitpath) + 'phet_cck_user_actions+sophistication_WITHPAUSE_more_circuit_info.txt',index_col=False)
# dfx = pd.read_csv('C:\Users\Sarah\Documents\git\Phet-log-analyzer\cck\\raw_data_parsing_check\phet_cck_user_actions+sophistication_WITHPAUSE_more_circuit_info.txt',index_col=False)
df["student"] = df["student"].astype('category')
df["Family"]=df["Family"].str.capitalize()

df_scores = pd.read_csv(data_path + 'MATCHING_phet_cck_user_data_anonymized.txt')
df_scores["student"] = df_scores["student"].astype('category')

In [3]:
CUT_OFF_SEQ_USE = 0.25

Let's get the sequences by students in blocks of actions.

## Here is how we get sequences 

In [66]:
converter =  {"Construct" : "C",
"Interface" : "I",
"Pause" : "P",
"Test_basic" : "Tb",
"Test_basic_default" : "Tbd",
"Test_basic_nonContactAmmeter" : "Tba",
"Test_basic_nonContactAmmeter_default" : "Tbad",
"Test_basic_nonContactAmmeter_not" : "Tban",
"Test_basic_not" : "Tbn",
"Test_basic_voltmeter" : "Tbv",
"Test_basic_voltmeter_default" : "Tbvd",
"Test_basic_voltmeter_not" : "Tbvn",
"Test_complex" : "Tc",
"Test_complex_default" : "Tcd",
"Test_complex_nonContactAmmeter" : "Tca",
"Test_complex_nonContactAmmeter_default" : "Tcad",
"Test_complex_nonContactAmmeter_not" : "Tcan",
"Test_complex_not" : "Tcn",
"Test_complex_seriesAmmeter" : "Tc",
"Test_complex_seriesAmmeter_default" : "Tcd",
"Test_complex_seriesAmmeter_not" : "Tcn",
"Test_complex_voltmeter" : "Tcv",
"Test_complex_voltmeter_default" : "Tcvd",
"Test_complex_voltmeter_not" : "Tcvn",
"Test_other" : "To",
"Test_other_default" : "Tod",
"Test_other_nonContactAmmeter" : "Toa",
"Test_other_nonContactAmmeter_default" : "Toad",
"Test_other_nonContactAmmeter_not" : "Toan",
"Test_other_not" : "Ton",
"Test_other_voltmeter_not" : "Tovn",
"Test_other_voltmeter" : "Tov",
"Test_other_voltmeter_default" : "Tovd",
"Test_simple" : "Ts",
"Test_simple_default" : "Tsd",
"Test_simple_nonContactAmmeter" : "Tsa",
"Test_simple_nonContactAmmeter_default" : "Tsad",
"Test_simple_nonContactAmmeter_not" : "Tsan",
"Test_simple_not" : "Tsn",
"Test_simple_voltmeter" : "Tsv",
"Test_simple_voltmeter_default" : "Tsvd",
"Test_simple_voltmeter_not" : "Tsvn",
    }



def get_blocks_withTime_new(df, students, as_list = True, ignore = [], start = True):
    '''gets blocks of sequences for a list of students'''
    def convert(action,ignore):
        return converter[action]
    
    if start:
        if add_spaces:
            blocks = {student:'S ' for student in students}
        else:
            blocks = {student:'S' for student in students}
    else:
        blocks = {student:'' for student in students}

    time_coords = {student:[] for student in students}
    for student in students:
        sequence =  list(df[df['student']==student]['Family'])
        time_stamps =  list(df[df['student']==student]['Time Stamp'])
        time_stamps = (time_stamps - min(time_stamps))/1000.  #human readable seconds
        time_coord=[]  #coordinate array for broken bar plot, takes array of (start time, duration)
        p = re.compile(r'([A-Z][a-z]{0,3})\1*')  #this regex finds all action blocks of length 1+
        #print ''.join([convert(action,ignore) for action in sequence])
        #print time_stamps
        #use finditer to return a sequence of matches as an iterator
        previous_start = 0
        for match in p.finditer(''.join([convert(action,ignore) for action in sequence])):
            ind = match.span()  #this gives start and end of matched block
            #for matches of action denoted by more than 1 letter, need to correct the span
            ind = (previous_start, previous_start + (ind[1]-ind[0])/len(set(match.group())))
            previous_start = ind[1]
            #print match.group(), ind
            if ind[1] >= len(time_stamps):  #block location offset from real index by 1
                duration = time_stamps[ind[1]-1] - time_stamps[ind[0]]  #time duration of block
                #print time_stamps[ind[1]-1], time_stamps[ind[0]]  #time duration of block
            else:
                duration = time_stamps[ind[1]] - time_stamps[ind[0]]
                #print time_stamps[ind[1]] , time_stamps[ind[0]]
            time_coord.append((time_stamps[ind[0]],duration))
            #print match.group(), match.span(), duration
        #actual regex that converts block of similar actions to just one action
        block = re.sub(r'([A-Z][a-z]{0,3})\1+', r'\1',''.join([convert(action,ignore) for action in sequence]))
        if as_list:            
            list_block = block[0] + ''.join([' ' + c if c.isupper() else c for c in block[1:]])
            blocks[student] = list_block.split(' ')
            time_coords[student] = time_coord
        else:
            blocks[student] += block
            time_coords[student] = time_coord
    return blocks, time_coords

In [67]:
students = [94792123,10942128]
blocks, time_coords =  get_blocks_withTime_new(df,students,start=False)
print blocks
print time_coords


{10942128: ['C', 'P', 'C', 'I', 'Tc', 'C', 'Tc', 'C', 'P', 'C', 'P', 'C', 'I', 'C', 'P', 'C', 'P', 'C', 'P', 'C', 'P', 'C', 'I', 'Tc'], 94792123: ['C', 'I', 'Tc', 'C', 'Tc', 'P', 'C', 'P', 'Tb', 'I', 'C', 'Tb', 'C', 'P', 'C', 'P', 'C', 'Tb', 'C', 'P', 'C', 'I', 'C', 'I', 'C', 'Tc', 'C', 'Ts', 'C', 'P', 'C', 'Tc', 'I', 'C', 'P', 'C', 'Tb', 'Ts', 'Tc', 'C', 'Tc', 'Tb', 'Ts', 'Tc', 'C', 'Ts', 'P', 'Tb', 'P', 'Ts', 'P', 'Tc', 'P', 'Tc', 'P', 'C', 'P', 'Ts', 'C', 'P', 'C']}
{10942128: [(0.0, 73.647000000000006), (73.647000000000006, 54.765000000000001), (128.41200000000001, 3.2980000000000018), (131.71000000000001, 4.3449999999999989), (136.05500000000001, 17.879999999999995), (153.935, 83.835999999999984), (237.77099999999999, 14.786000000000001), (252.55699999999999, 50.125000000000028), (302.68200000000002, 106.76400000000001), (409.44600000000003, 179.08399999999995), (588.52999999999997, 70.456000000000017), (658.98599999999999, 12.988000000000056), (671.97400000000005, 2.6889999999999

In [86]:
def get_frequencies(blocks, shortest=3, longest=11):
    frequencies = {student:Counter() for student in blocks.keys()}
    for student,sequence in blocks.iteritems():
        for seq_length in range(shortest, longest+1):  # loops through different possible sequence lengths
            frequencies[student] += Counter(''.join(sequence[i:i+seq_length]) for i in range(len(sequence)-seq_length+1))  # counts string matches for every string of the current length
    return frequencies

In [87]:
frequencies = get_frequencies(blocks, shortest=2, longest=4)
#Count number of time sequences occurs accross all students
sum_frequencies = sum(frequencies.values(), Counter())
#Count number of students that conducted that frequency
counts_frequencies = Counter({f:sum([ 1 if f in freq else 0 for freq in frequencies.values()]) for f in list(sum_frequencies)})

In [88]:
#We need to know, for each student, what is the index of action that starts each bin
action_bins = {}
B = 5
for student in students:
    action_indices = [None for i in range(1,B)]
    total_time = sum(time_coords[student][-1])
    time_of_bins = [total_time*i/B for i in range(1,B)]
    for i,(t,duration) in enumerate(time_coords[student]):
        for j,b in enumerate(time_of_bins):
            if t <= b:
                action_indices[j] = i+1
    action_indices.append(len(time_coords[student])) #add last index
    action_indices.insert(0,0)
    #Transform bins in tuples of ranges
    action_bins[student] = [(action_indices[i],action_indices[i+1]) for i in range(0,B)]

print action_bins

{10942128: [(0, 9), (9, 11), (11, 17), (17, 19), (19, 24)], 94792123: [(0, 15), (15, 30), (30, 36), (36, 55), (55, 61)]}


In [92]:
def get_frequencies_by_bin(blocks, B, action_bins,shortest=3, longest=11):
    frequencies = {student:[Counter() for i in range(B)] for student in blocks.keys()}
#     def fix_seq(sequence,i,seq_length,end_action):
#         if sequence[i] == sequence[i].upper() :    #check that first letter in seq is UPPER
#             #extend pattern until next digit is not a lower case, and that the length of the seq is actually seq_length, 
#             j = i
#             new_seq = sequence[i:i+seq_length]
#             while len([c for c in new_seq if c.isupper()]) < seq_length and sequence[i+seq_length+1].isupper():
#                 j += 1
#                 new_seq = sequence[i:j+seq_length]
                
# #             print i, seq_length, sequence[i:i+seq_length], new_seq
#             return new_seq
        
    for student,sequence in blocks.iteritems():
        for seq_length in range(shortest, longest+1):  # loops through different possible sequence lengths
            for j,(start_action,end_action) in enumerate(action_bins[student]): 
                #since we want to find sequence THAT START in bin, we remove the parts of the sequence that fall in previous bins
                portion_of_sequence = sequence[start_action:end_action]
                frequencies[student][j] += Counter(''.join(portion_of_sequence[i:i+seq_length]) for i in range(len(portion_of_sequence)-seq_length+1))  # counts string matches for every string of the current length
    return frequencies


frequencies_by_bin = get_frequencies_by_bin(blocks, B, action_bins,shortest=2, longest=2)
print blocks
print frequencies_by_bin

{10942128: ['C', 'P', 'C', 'I', 'Tc', 'C', 'Tc', 'C', 'P', 'C', 'P', 'C', 'I', 'C', 'P', 'C', 'P', 'C', 'P', 'C', 'P', 'C', 'I', 'Tc'], 94792123: ['C', 'I', 'Tc', 'C', 'Tc', 'P', 'C', 'P', 'Tb', 'I', 'C', 'Tb', 'C', 'P', 'C', 'P', 'C', 'Tb', 'C', 'P', 'C', 'I', 'C', 'I', 'C', 'Tc', 'C', 'Ts', 'C', 'P', 'C', 'Tc', 'I', 'C', 'P', 'C', 'Tb', 'Ts', 'Tc', 'C', 'Tc', 'Tb', 'Ts', 'Tc', 'C', 'Ts', 'P', 'Tb', 'P', 'Ts', 'P', 'Tc', 'P', 'Tc', 'P', 'C', 'P', 'Ts', 'C', 'P', 'C']}
{10942128: [Counter({'CP': 2, 'TcC': 2, 'CI': 1, 'ITc': 1, 'PC': 1, 'CTc': 1}), Counter({'CP': 1}), Counter({'CP': 2, 'PC': 1, 'CI': 1, 'IC': 1}), Counter({'CP': 1}), Counter({'PC': 1, 'CI': 1, 'CP': 1, 'ITc': 1})], 94792123: [Counter({'PC': 2, 'CP': 2, 'CI': 1, 'ITc': 1, 'TcP': 1, 'PTb': 1, 'TbI': 1, 'TcC': 1, 'CTb': 1, 'CTc': 1, 'IC': 1, 'TbC': 1}), Counter({'CI': 2, 'PC': 2, 'IC': 2, 'CP': 2, 'CTs': 1, 'TsC': 1, 'CTb': 1, 'CTc': 1, 'TbC': 1, 'TcC': 1}), Counter({'PC': 1, 'IC': 1, 'CP': 1, 'CTc': 1, 'TcI': 1}), Counter

## Here is how to select certain sequences

In [95]:
## Clean up sequences with count less than N
N = int(CUT_OFF_SEQ_USE*len(blocks.keys()))
remove_rare_frequencies(counts_frequencies,N)
#keep_frequencies_with(counts_frequencies,keep='P')
frequencies = clean_student_frequencies(counts_frequencies, frequencies)
counts_frequencies.most_common(10)

[('CIC', 2),
 ('CPC', 2),
 ('TcCTc', 2),
 ('CITc', 2),
 ('ITcC', 2),
 ('ICP', 2),
 ('TcC', 2),
 ('CTc', 2),
 ('CI', 2),
 ('CPCP', 2)]

### Let's see what sequences occur often for both activities

In [ ]:
blocks = get_blocks(df,get_students())
frequencies = get_frequencies(blocks, shortest=3, longest=10)
#Count number of time sequences occurs accross all students
sum_frequencies = sum(frequencies.values(), Counter())
#Count number of students that conducted that frequency
counts_frequencies = Counter({f:sum([ 1 if f in freq else 0 for freq in frequencies.values()]) for f in list(sum_frequencies)})

In [ ]:
counts_frequencies

## Let's compare sequences from students in the different conditions in Activity 1
Get all frequencies of length 3 to 10, cleaning them up by removing those common to all students and the rare ones.

In [ ]:
summed_seqs, seq_scaff, seq_not = mega_process(activity='a1', level='scaffolding', value1='scaff', value2='not', shortest=3, longest=10)
difference(seq_scaff,seq_not,N=10)
seq_scaff

We can now look at the difference in number of students that conduct sequences in both groups.

In [ ]:
## Does this difference persist in activity 2?

In [ ]:
summed_seqs, seq_scaff, seq_not = mega_process(activity='a2', level='scaffolding', value1='scaff', value2='not', shortest=3, longest=10)
difference(seq_scaff,seq_not,N=10)

## Let's compare sequences from students from different clusters
In activity 1

In [ ]:
summed_seqs, seq_know, seq_att = mega_process(activity='a1', level='Clustergroups', value1='knowledge', value2='attitude', shortest=3, longest=10)
difference(seq_know+seq_know,seq_att,N=10)

**Note**: Here we are doubling the number of counts for the knowledge group since it's half the size

In activity 2

In [ ]:
summed_seqs, seq_know, seq_att = mega_process(activity='a2', level='Clustergroups', value1='knowledge', value2='attitude', shortest=3, longest=10)
difference(seq_know+seq_know,seq_att,N=10)

**Note**: Here we are doubling the number of counts for the knowledge group since it's half the size

## Let's compare sequences from students with high and low learning gain

First we need to split students by learning gains in each activity

In [ ]:
def label_learning (row,column):
    if row[column] >= median_learning: return 'high'
    else: return 'low'

df_scores['learning1score'] = float(df_scores["post t1"] - df_scores["pre"])/float(df_scores["pre"])
median_learning = np.median(df_scores['learning1score'])
df_scores['learning1'] = df_scores.apply (lambda row: label_learning (row,"learning1score"),axis=1)

df_scores['learning2score'] = float(df_scores["post t2"] - df_scores["pre"])/float(df_scores["pre"])
median_learning = np.median(df_scores['learning2score'])
df_scores['learning2'] = df_scores.apply (lambda row: label_learning (row,"learning2score"),axis=1)
# len(df_scores[(df_scores['learning1']=='high') & (df_scores['learning2']=='high')])

Now we look at the different in sequences in activity 1

In [ ]:
summed_seqs, seq_high, seq_low = mega_process(activity='a1', level='learning1', value1='high', value2='low', shortest=3, longest=10)
difference(seq_high,seq_low,N=10)



And activity 2

In [ ]:
summed_seqs, seq_high, seq_low = mega_process(activity='a2', level='learning2', value1='high', value2='low', shortest=3, longest=10)
difference(seq_high,seq_low,N=10)

## Well, one thing we are really interested in, is what happens before/after students pause. So let's look at those sequences, and compare low/high learners.

In [ ]:
# summed_seqs, seq_high, seq_low = mega_process(activity='a1', level='learning1', value1='high', 
#                                               value2='low', shortest=3, longest=10, keep = "P")
# difference(seq_high,seq_low,N=10)

In [ ]:
# summed_seqs, seq_high, seq_low = mega_process(activity='a2', level='learning2', value1='high', 
#                                               value2='low', shortest=3, longest=10, keep = "P")
# difference(seq_high,seq_low,N=10)

#### Now let's pick a sequence visualize how much it's used by students

In [ ]:
def plot_seq_gains(activity, seqs):
    df2 = df[df.Activity == activity]
    if activity == 'a1': score_column = "learning1score"
    else: score_column = "learning2score"
        
    blocks = get_blocks(df2,get_students())
    frequencies = get_frequencies(blocks, shortest=3, longest=6)
    sum_frequencies = sum(frequencies.values(), Counter())
    counts_frequencies = Counter({f:sum([ 1 if f in freq else 0 for freq in frequencies.values()]) for f in list(sum_frequencies)})
    totals = {student:float(sum(count.values())) for student,count in frequencies.iteritems()}

    fig, axs = plt.subplots(nrows=len(seqs),ncols=1,figsize=(15,3*len(seqs)))
    fig.suptitle('Normalize frequency of usage of sequences versus learning gain. Students are colored depending on their scaffolding.',fontsize=16)
    for seq,ax in zip(seqs,axs):
        counts = keep_frequencies_with(counts_frequencies,keep=seq)
        cleaned_frequencies = clean_student_frequencies(counts, frequencies)

        f1 = select_frequencies(cleaned_frequencies,"scaffolding","scaff")
        f2 = select_frequencies(cleaned_frequencies,"scaffolding","not")

        total1 = {student:float(sum(count.values()))/totals[student] for student,count in f1.iteritems()}
        values1 = {k:(v, float(df_scores[df_scores["student"]==k][score_column]) ) for k,v in total1.iteritems()}
        total2 = {student:float(sum(count.values()))/totals[student] for student,count in f2.iteritems()}
        values2 = {k:(v, float(df_scores[df_scores["student"]==k][score_column]) ) for k,v in total2.iteritems()}
        x,y = zip(*values1.values())
        rec1 = ax.scatter(x,y, c='r',s=100, alpha=0.4)
        x,y = zip(*values2.values())
        rec2 = ax.scatter(x,y, c='b',s=100, alpha=0.4)
        ax.axhline(0, color='darkgrey')
        ax.set_ylabel(seq)
        ax.legend((rec1, rec2), ('Scaffolded', 'Not scaffolded'))
        
#         ax.set_xlabel('Normalized frequency of usage')
#         ax.set_ylabel('Learning gain in activity 1 (post-pre)')
#         ax.set_title('Normalize frequency of usage of {0} versus learning gain. Students are colored depending on their scaffolding.'.format(seq))

### Want to make a matrix where rows are sequences, columns students, cells represent number of times sequence was performed by student

In [ ]:
df2 = df[df.Activity == 'a1']
blocks = get_blocks(df2,get_students())
frequencies = get_frequencies(blocks, shortest=3, longest=6)
sum_frequencies = sum(frequencies.values(), Counter())
counts_frequencies = Counter({f:sum([ 1 if f in freq else 0 for freq in frequencies.values()]) for f in list(sum_frequencies)})
totals = {student:float(sum(count.values())) for student,count in frequencies.iteritems()}

students, sequences = frequencies.keys(), sum_frequencies.keys()
A = np.zeros((len(students), len(sequences)))
A.shape

for st,seqs in frequencies.iteritems():
    i = students.index(st)
    for sq,count in seqs.iteritems():
        j = sequences.index(sq)
        A[i][j] = count

We normalize the matrix (in reality the right way to normalize is to divide by number of blocks of sequences per student. Right now, the number of sequences per students depends on how diverse their set of seqs is)

In [ ]:
B = np.zeros((len(students), len(sequences)))
for i,row in enumerate(A):
    B[i,:] = A[i,:]/len(blocks[students[i]])

We calculate correlations with learning outcomes for each sequence

In [ ]:
gains = [round(float(df_scores[df_scores["student"]==student]['learning1score']),5) for student in students]

from scipy.stats import pearsonr
correlation_measure = pearsonr
corr = []
for sq in sequences:
    j = sequences.index(sq)
    if np.count_nonzero(B[:,j]) > 10:
        c = correlation_measure(B[:,j],gains)
        corr.append((sq, c[0], c[1]))    

Now let's keep only the significant sequences.

**Note**: we still need to do some boostrapping and permutation test to confirm that these are significant results.

In [ ]:
sig_corr = []
for s,c,p in corr:
    p = p/2.0
    if p <= 0.05 and abs(c) > 0.25:
        sig_corr.append((s,c,p))
        print s,c,p        

In [ ]:
plot_seq_gains('a1',zip(*sig_corr)[0])

Clearly, how much students did an action doesn't matter as much as who did it at least once (or more?)
Let's repeat the analysis using # of students who did the sequence instead of frequency.

In [ ]:
df2 = df[df.Activity == 'a1']
blocks = get_blocks(df2,get_students())
frequencies = get_frequencies(blocks, shortest=3, longest=6)
sum_frequencies = sum(frequencies.values(), Counter())
counts_frequencies = Counter({f:sum([ 1 if f in freq else 0 for freq in frequencies.values()]) for f in list(sum_frequencies)})
totals = {student:float(sum(count.values())) for student,count in frequencies.iteritems()}

students, sequences = frequencies.keys(), sum_frequencies.keys()
A = np.zeros((len(students), len(sequences)))
A.shape


for st,seqs in frequencies.iteritems():
    i = students.index(st)
    for sq,count in seqs.iteritems():
        j = sequences.index(sq)
        if count >= 1: x =1
        else: x = 0
        A[i][j] = x

In [ ]:
from scipy.stats import ttest_ind
gains = np.array([round(float(df_scores[df_scores["student"]==student]['learning1score']),5) for student in students])

tests = []
seq_gains = {}
for sq in sequences:
    j = sequences.index(sq)
    a = gains[np.where(A[:,j]==1)] #learning gains of student who used sequence
    b = gains[np.where(A[:,j]==0)] #learning gains of student who didn't
    t = ttest_ind(a, b, equal_var=False)
    tests.append((sq, t[0], t[1]))
    seq_gains[sq] = (a,b)

In [ ]:
sig_tests = []
print "Sequence, # students/{0}, t-test, p-value".format(len(students))
for s,t,p in tests:
    p = p/2.0
    if p <= 0.05:
        n = len(seq_gains[s][0]) #number of students who did the sequence
        if n > 10 and n < 86: # and 'P' in s:
            sig_tests.append((s,n,t,p))

# order by ttest value
results = sorted(sig_tests, key=lambda x: x[2])
for s,n,t,p in results:
    print "{0} \t\t {1}\t {2} \t {3}".format(s, n, round(t,2), round(p,3))

In [ ]:
plt.figure()
seq = 'ROPT'
means = [np.mean(seq_gains[seq][0]),np.mean(seq_gains[seq][1])]
stdes = [np.std(seq_gains[seq][0]),np.std(seq_gains[seq][1])]
plt.boxplot(seq_gains[seq])
plt.ylabel('Average learning gain')
plt.xticks(range(1,3),[str(len(seq_gains[seq][0])) +' did it', str(len(seq_gains[seq][1])) + ' didn\'t'])
plt.title('Learning gains depending on use of sequence '+seq)
plt.grid()